# Lab 7 DATA SCIENCE Guia Spark


## Creacion de la aplicacion

In [2]:
from pyspark.sql import SparkSession

# ruta DENTRO DEL CONTENEDOR
working_dir = "/opt/app/working_dir/"

spark = (SparkSession.builder
         .appName("Lab7-RDDs")
         .master("local[*]")
         .getOrCreate())
sc = spark.sparkContext

spark, sc.master, working_dir


/usr/local/lib/python3.11/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/26 04:34:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


(<pyspark.sql.session.SparkSession at 0x7f38d3251090>,
 'local[*]',
 '/opt/app/working_dir/')

## Exploracion basica

In [4]:
rdd = sc.textFile(working_dir + "constitution.txt")
print("Ejemplo (3 líneas):", rdd.take(3))
print("Número de líneas:", rdd.count())


Ejemplo (3 líneas): ['We the People of the United States, in Order to form a more perfect ', 'Union, establish Justice, insure domestic Tranquility, provide for the ', 'common defence, promote the general Welfare, and secure the Blessings of ']
Número de líneas: 649


Usando el count podemos definir la cantidad de líneas en este caso tenemos 649 líneas sin definir el split


## Word Count

In [5]:
splitted_lines = rdd.map(lambda line: line.split(' '))

print("Ejemplo (3 elementos):", splitted_lines.take(3))
print("Tipo del primer elemento:", type(splitted_lines.take(1)[0]))
print("¿Cuántos elementos tiene splitted_lines?:", splitted_lines.count())  # ¿qué te da?


Ejemplo (3 elementos): [['We', 'the', 'People', 'of', 'the', 'United', 'States,', 'in', 'Order', 'to', 'form', 'a', 'more', 'perfect', ''], ['Union,', 'establish', 'Justice,', 'insure', 'domestic', 'Tranquility,', 'provide', 'for', 'the', ''], ['common', 'defence,', 'promote', 'the', 'general', 'Welfare,', 'and', 'secure', 'the', 'Blessings', 'of', '']]
Tipo del primer elemento: <class 'list'>
¿Cuántos elementos tiene splitted_lines?: 649


No cumple aún con el propósito porque incluso sigue contando las líneas y en el output podemos ver que esta agrupando incorrectamente el split. un RDD donde cada elemento es una lista de palabras, no una palabra.

In [8]:
words_rdd = rdd.flatMap(lambda line: line.split(' '))

print("Muestra de palabras:", words_rdd.take(10))
print("Total de 'palabras':", words_rdd.count())


Muestra de palabras: ['We', 'the', 'People', 'of', 'the', 'United', 'States,', 'in', 'Order', 'to']
Total de 'palabras': 8435


Con flatMap cada elemento del RDD pasó a ser una palabra individual

In [9]:
words_rdd_clean_split_space = (rdd
    .flatMap(lambda line: line.split(' '))
    .filter(lambda w: w != ""))

print("Tokens tras filtrar vacíos:", words_rdd_clean_split_space.count())


Tokens tras filtrar vacíos: 7623


Se convirtió cada línea en palabras usando flatMap y separando por espacio, entonces el valor impreso corresponde al total de tokens reales del documento sin vaciios.

In [11]:
longest_raw = words_rdd_clean_split_space.reduce(
    lambda a, b: a if len(a) > len(b) else b
)
longest_raw, len(longest_raw)


('Representatives,', 16)

comparando longitudes el resultado todavía puede incluir signos de puntuación pero se considera la mas larga por el momento

### Configuracion con isalnum()

In [13]:
words_clean = (words_rdd_clean_split_space
               .filter(lambda w: w.isalnum())
               .map(lambda w: w.lower()))

longest_clean = words_clean.reduce(
    lambda a, b: a if len(a) > len(b) else b
)
longest_clean, len(longest_clean)


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


('constitutionally', 16)

Ahora sí sin usar la coma en representatives ni considerarlas siendo "minusculas" convertidas pero sin ningun caracter especial podemos si encontrar la palabra mas larga

## KEY-VALUE RDDs

In [16]:
# Convirtiendo a rdd a keyvalues: 

base_words_rdd = globals().get("words_rdd", None) or words_rdd_clean_split_space

keyval_rdd = base_words_rdd.map(lambda w: (w, 1))
wordcount = keyval_rdd.reduceByKey(lambda a, b: a + b)
print("wordcount:", wordcount.take(5))


wordcount: [('We', 2), ('of', 493), ('United', 85), ('States,', 55), ('Order', 1)]


Esto permite obtener el conteo individual por palabra de forma eficiente en paralelo.

In [17]:
sorted_wc = (wordcount
             .map(lambda x: (x[1], x[0]))  
             .sortByKey(ascending=False))  

top5 = sorted_wc.take(5)
top5


[(812, ''), (662, 'the'), (493, 'of'), (293, 'shall'), (256, 'and')]

In [19]:
wordcount_clean = (words_clean
                   .map(lambda w: (w, 1))
                   .reduceByKey(lambda a, b: a + b))

top5_clean = (wordcount_clean
              .map(lambda x: (x[1], x[0]))
              .sortByKey(ascending=False)
              .take(5))
top5_clean


[(726, 'the'), (493, 'of'), (293, 'shall'), (262, 'and'), (201, 'to')]

In [23]:
stopwords = {
    'the','of','and','to','in','a','that','is','for','on','with','as','by','be','are','was','were',
    'this','it','or','an','from','at','which','but','not','have','has','had','shall','state','may','any','no','all','such' 
}

wordcount_nostop = (words_clean
                    .filter(lambda w: w not in stopwords)
                    .map(lambda w: (w, 1))
                    .reduceByKey(lambda a, b: a + b))

top5_nostop = (wordcount_nostop
               .map(lambda x: (x[1], x[0])) 
               .sortByKey(ascending=False)
               .take(5))
top5_nostop


[(85, 'united'),
 (72, 'president'),
 (47, 'states'),
 (39, 'congress'),
 (34, 'amendment')]

Por el simple interes de conocer se ha agregado a los stopwords tambien "no" ya que no necesariamente es un stopword en el sentido que no es conector directamente, y state porque ya esta states en plural dentro de los "ganadores". 

---

En cuanto a comentarios sobre spark puede ser un poco mas abstracto no necesariamente mas como, cómo? tenemos que hacer exactamente algo sino que y ya. Interesante porque ayuda a trabajar con archivos grandes.  